# Pokemon TCG API has a [Python SDK](https://github.com/PokemonTCG/pokemon-tcg-sdk-python)
By installing this SDK, I can speed up the development process. This eliminates a lot of work interfacing nicely with the API.

### Configure REST Client with API Key


In [1]:
from pokemontcgsdk import RestClient
import yaml as yaml

with open('../sensitive_config/api_config.yaml','r') as f:
    config = yaml.safe_load(f)

RestClient.configure(config['api_key'])

### Test Queries

In [5]:
from pokemontcgsdk import Card
from pokemontcgsdk import Set
from pokemontcgsdk import Type
from pokemontcgsdk import Supertype
from pokemontcgsdk import Subtype
from pokemontcgsdk import Rarity

### Testing Static Dimension Tables

In [ ]:
supertypes = Supertype.all()
supertypes

In [9]:
subtypes = Subtype.all()
subtypes[0:4]

['ACE SPEC', 'Ancient', 'BREAK', 'Baby']

In [10]:
types = Type.all()
types[0:4]

['Colorless', 'Darkness', 'Dragon', 'Fairy']

In [13]:
rarities = Rarity.all()
rarities[-4:]

['Special Illustration Rare',
 'Trainer Gallery Rare Holo',
 'Ultra Rare',
 'Uncommon']

In [27]:
import json
from pathlib import Path

data_path = Path().cwd().parent / 'data'
raw_data_path = data_path / 'raw'

In [29]:
with open(raw_data_path / 'rarities.json', 'w') as f:
    json.dump(rarities, f, indent=4)

with open(raw_data_path / 'types.json', 'w') as f:
    json.dump(types, f, indent=4)

with open(raw_data_path / 'subtypes.json', 'w') as f:
    json.dump(subtypes, f, indent=4)

with open(raw_data_path / 'supertypes.json', 'w') as f:
    json.dump(supertypes, f, indent=4)

### Testing Set API

In [ ]:
import dataclasses as dc
from datetime import datetime

In [32]:
sets_catalog = Set.all()
# Convert list of Dataclass objects to list of dictionaries
sets_catalog_as_dict: list[dict] = [dc.asdict(card_set) for card_set in sets_catalog]
sets_catalog_as_dict[0]

{'id': 'base1',
 'images': {'symbol': 'https://images.pokemontcg.io/base1/symbol.png',
  'logo': 'https://images.pokemontcg.io/base1/logo.png'},
 'legalities': {'unlimited': 'Legal', 'expanded': None, 'standard': None},
 'name': 'Base',
 'printedTotal': 102,
 'ptcgoCode': 'BS',
 'releaseDate': '1999/01/09',
 'series': 'Base',
 'total': 102,
 'updatedAt': '2022/10/10 15:12:00'}

In [ ]:
# Add metadata around the pulled data
def wrap_with_metadata(data: list[dict]) -> dict:
    get_current_time = lambda: datetime.today().strftime('%m/%d/%Y %H:%M:%S')
    wrapped_data = {'pulled_on':get_current_time(),
                    'count':len(data),
                    'data':data}
    return wrapped_data
    

In [57]:
with open(raw_data_path / 'sets_catalog.json', 'w') as f:
    json.dump(wrap_with_metadata(sets_catalog_as_dict), f, indent=4)

### Testing Card API as Full Catalog

In [39]:
import time
tic = time.time()
cards_catalog = Card.all()
toc = time.time()
print(f'Query for all cards executed in {(toc-tic)/60.0} minutes')

Query for all cards executed in 8.990885547796886 minutes


In [60]:
cards_catalog_as_dict: list[dict] = [dc.asdict(card) for card in cards_catalog]
cards_catalog_as_dict[0]['name'], cards_catalog_as_dict[-1]['name']

('Aggron', 'Spiky Energy')

In [62]:
with open(raw_data_path / 'cards_catalog.json', 'w') as f:
    json.dump(wrap_with_metadata(cards_catalog_as_dict), f, indent=4)

with open(raw_data_path / 'cards_sample.json', 'w') as f:
    json.dump(wrap_with_metadata([cards_catalog_as_dict[0]]), f, indent=4)

### Testing Card API as Partial Fields Catalog

In [86]:
price_capture_fields_of_interest = ['name','id','tcgplayer','legalities']
# This is because the Dacite python package that QueryBuilder uses creates a Card dataclass from the response dictionary
# So even if the '?select=name,id,...' from price_capture_fields_of_interest is executed with a correct API response,
# there are non Optional type fields in the Card dataclass that require definition or an error occurs.
mandatory_other_fields = ['images','number','supertype', 'set'] 
fields_subset = price_capture_fields_of_interest + mandatory_other_fields
foi_string = ','.join(fields_subset)

In [80]:
data = Card.where(q='id:hgss4-1', select=foi_string)

In [87]:
data_as_dict: list[dict] = [dc.asdict(datum) for datum in data]
subset_fields_of_interest = lambda data, fields: [{field:datum[field] for field in fields} for datum in data] # assumes fields of interest are top-level only
subsetted_data_as_dict = subset_fields_of_interest(data_as_dict, price_capture_fields_of_interest)

In [88]:
subsetted_data_as_dict

[{'name': 'Aggron',
  'id': 'hgss4-1',
  'tcgplayer': {'url': 'https://prices.pokemontcg.io/tcgplayer/hgss4-1',
   'updatedAt': '2025/04/01',
   'prices': {'normal': None,
    'holofoil': {'low': 2.0,
     'mid': 2.74,
     'high': 4.04,
     'market': 2.33,
     'directLow': None},
    'reverseHolofoil': {'low': 2.1,
     'mid': 4.0,
     'high': 500.0,
     'market': 2.35,
     'directLow': None},
    'firstEditionHolofoil': None,
    'firstEditionNormal': None}},
  'legalities': {'unlimited': 'Legal', 'expanded': None, 'standard': None}}]

In [89]:
### Now Testing a Full-catalog Price Update

In [90]:
tic = time.time()
cards_catalog_price_update = Card.where(select=foi_string)
toc = time.time()
print(f'Query for all cards executed in {(toc-tic)/60.0} minutes')

Query for all cards executed in 9.137225449085236 minutes


In [91]:
cards_catalog_price_update_as_dict: list[dict] = [dc.asdict(card) for card in cards_catalog_price_update]
subsetted_cards_catalog_price_update = subset_fields_of_interest(cards_catalog_price_update_as_dict, price_capture_fields_of_interest)

In [94]:
with open(raw_data_path / 'cards_catalog_update.json', 'w') as f:
    json.dump(wrap_with_metadata(subsetted_cards_catalog_price_update), f, indent=4)